In [ ]:
import random

data_dict = {f"seed_{i}":{} for i in range(0, 10)}

for seed in range(0, 10):
    # 기존 구조: train_set: data['partition]['Train_Set], valid_set: data['partition']['Validation_Set']
    # Test Set은 사용 x
    # 모든 데이터 합친 후 6:2:2
    file_list = list(data['partition']['Train_Set'].keys()) + list(data['partition']['Validation_Set'].keys())

    train_len = int(len(file_list) * 0.6)
    valid_len = int(len(file_list) * 0.2)
    test_len = int(len(file_list) * 0.2)
    
    train_samples = random.sample(file_list, train_len) # 랜덤하게 뽑기
    file_list = [file for file in file_list if not file in train_samples] # 안뽑힌 것들만 남기기
    
    valid_samples= random.sample(file_list, valid_len) # 랜덤하게 뽑기
    file_list = [file for file in file_list if not file in valid_samples] # 안뽑힌 것들만 남기기
    
    test_samples = file_list # 나머진 테스트 셋
    
    seed_data_dict = {"Train_Set":train_samples,
                      "Validation_Set":valid_samples,
                      "Test_Set":test_samples
                      }
    
    data_dict[f'seed_{seed}'] = seed_data_dict
    
data_dict

In [ ]:
import random
import json

# 파일 경로
file_path = '/userHome/userhome2/dahyun/glow-tts/filelists/ljs_audio_text.txt'

# 데이터 파일 읽기
with open(file_path, 'r') as f:
    data = f.readlines()

# 함수: 데이터를 random seed를 사용하여 8:2로 나누고, 그 중 75%는 학습, 25%는 검증 데이터로 나눔
def split_data(data, seed):
    random.seed(seed)
    random.shuffle(data)
    
    # 8:2 split (Train+Validation:Test)
    split_index = int(len(data) * 0.8)
    train_val_data = data[:split_index]
    test_data = data[split_index:]
    
    # Train:Validation = 75:25 split
    train_split_index = int(len(train_val_data) * 0.75)
    train_data = train_val_data[:train_split_index]
    validation_data = train_val_data[train_split_index:]
    
    return train_data, validation_data, test_data

# 10개의 random seed에 따라 데이터를 나누어 저장할 딕셔너리
split_results = {}

# 10번 반복하여 seed 0~9로 데이터 분할
for seed in range(10):
    train_data, validation_data, test_data = split_data(data, seed)
    split_results[f'seed_{seed}'] = {
        'Train_Set': train_data,
        'Validation_Set': validation_data,
        'Test_Set': test_data
    }
    
    # 각 시드별로 train, val, test 데이터 개수 출력
    print(f"Seed {seed}:")
    print(f"  Train_Set: {len(train_data)} samples")
    print(f"  Validation_Set: {len(validation_data)} samples")
    print(f"  Test_Set: {len(test_data)} samples")
    print()

# 결과를 json 파일로 저장
output_file = '/userHome/userhome2/dahyun/glow-tts/filelists/glowtts_new_data_splits.json'
with open(output_file, 'w') as json_file:
    json.dump(split_results, json_file, indent=4)

print(f"Data splits saved to {output_file}")
